In [1]:
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
import itertools

from keras.models import load_model
from keras.utils import np_utils

import tensorflow as tf
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb
import time

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
#         visible_device_list="1", 
        allow_growth=True
    )
)

set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load data
df = pd.concat([
    pd.read_csv('../data/feature_selection_positive.csv', index_col=0),
    pd.read_csv('../data/decomp_pos.csv', index_col=0).drop('Subclass', axis=1)
], axis=1)

In [3]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [4]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [5]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [6]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# append accuracy to list
t = []
t.append(clf.score(X_test, y_test))
pickle.dump(clf, open('../model/rf_pos_fs+decomp.sav', 'wb'))

In [7]:
# setting params
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [8]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

pickle.dump(model, open('../model/xgb_pos_fs+decomp.sav', 'wb'))
t.append(model.score(X_test, y_test))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
# define and fit 
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)
pickle.dump(gbm, open('../model/lgbm_pos_fs+decomp.sav', 'wb'))
t.append(gbm.score(X_test, y_test))

[1]	valid_0's multi_logloss: 1.29069
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.20999
[3]	valid_0's multi_logloss: 1.14184
[4]	valid_0's multi_logloss: 1.08348
[5]	valid_0's multi_logloss: 1.03123
[6]	valid_0's multi_logloss: 0.984326
[7]	valid_0's multi_logloss: 0.940083
[8]	valid_0's multi_logloss: 0.899747
[9]	valid_0's multi_logloss: 0.864059
[10]	valid_0's multi_logloss: 0.832391
[11]	valid_0's multi_logloss: 0.804595
[12]	valid_0's multi_logloss: 0.779628
[13]	valid_0's multi_logloss: 0.75554
[14]	valid_0's multi_logloss: 0.733512
[15]	valid_0's multi_logloss: 0.715433
[16]	valid_0's multi_logloss: 0.697284
[17]	valid_0's multi_logloss: 0.680813
[18]	valid_0's multi_logloss: 0.666624
[19]	valid_0's multi_logloss: 0.64985
[20]	valid_0's multi_logloss: 0.637678
[21]	valid_0's multi_logloss: 0.622767
[22]	valid_0's multi_logloss: 0.612299
[23]	valid_0's multi_logloss: 0.602638
[24]	valid_0's multi_logloss: 0.592583
[25]	valid_0's mult

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [10]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_pos_fs+decomp.h5')
t.append(model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 51s 38ms/step - loss: 1.8712 - acc: 0.2560 - val_loss: 1.5267 - val_acc: 0.2262
Epoch 2/100
1340/1340 [==============================] - 2s 1ms/step - loss: 1.7937 - acc: 0.2694 - val_loss: 1.4197 - val_acc: 0.2798
Epoch 3/100
1340/1340 [==============================] - 2s 1ms/step - loss: 1.7008 - acc: 0.2545 - val_loss: 1.3748 - val_acc: 0.3006
Epoch 4/100
1340/1340 [==============================] - 1s 964us/step - loss: 1.6782 - acc: 0.2858 - val_loss: 1.3647 - val_acc: 0.3333
Epoch 5/100
1340/1340 [==============================] - 1s 693us/step - loss: 1.6381 - acc: 0.2866 - val_loss: 1.3531 - val_acc: 0.3155
Epoch 6/100
1340/1340 [==============================] - 1s 753us/step - loss: 1.5906 - acc: 0.3164 - val_loss: 1.3554 - val_acc: 0.3036
Epoch 7/100
1340/1340 [==============================] - 1s 1ms/step - loss: 1.5630 - acc: 0.3134 - val_loss: 1.3534 - val_acc: 0.3810


Epoch 61/100
1340/1340 [==============================] - 2s 2ms/step - loss: 0.5885 - acc: 0.7634 - val_loss: 0.9436 - val_acc: 0.6250
Epoch 62/100
1340/1340 [==============================] - 2s 2ms/step - loss: 0.5819 - acc: 0.7709 - val_loss: 0.9344 - val_acc: 0.6250
Epoch 63/100
1340/1340 [==============================] - 2s 2ms/step - loss: 0.5684 - acc: 0.7843 - val_loss: 0.9122 - val_acc: 0.6488
Epoch 64/100
1340/1340 [==============================] - 2s 1ms/step - loss: 0.5507 - acc: 0.7664 - val_loss: 0.9578 - val_acc: 0.6458
Epoch 65/100
1340/1340 [==============================] - 2s 1ms/step - loss: 0.5257 - acc: 0.7821 - val_loss: 1.0215 - val_acc: 0.6310
Epoch 66/100
1340/1340 [==============================] - 2s 1ms/step - loss: 0.5356 - acc: 0.7828 - val_loss: 1.0231 - val_acc: 0.6310
Epoch 67/100
1340/1340 [==============================] - 2s 2ms/step - loss: 0.4939 - acc: 0.8052 - val_loss: 0.9843 - val_acc: 0.6310
Epoch 68/100
1340/1340 [========================

In [11]:
f = []
f.append(t)

In [12]:
f

[[0.7797619047619048,
  0.8035714285714286,
  0.8511904761904762,
  0.6309523809523809]]

In [13]:
# load data
df = pd.concat([
    pd.read_csv('../data/feature_selection_negative.csv', index_col=0),
    pd.read_csv('../data/decomp_neg.csv', index_col=0).drop('Subclass', axis=1)
], axis=1)

In [14]:
df

,Subclass,HCD35_214.31,HCD35_151.08,HCD35_139.96,HCD35_178.03,HCD35_201.02,HCD35_131.0,HCD35_256.07,HCD35_189.09,HCD35_288.9,...,NMF_0_6,NMF_1_6,NMF_2_6,NMF_3_6,NMF_4_6,FAG_0_6,FAG_1_6,FAG_2_6,FAG_3_6,FAG_4_6
0,Carbohydrates and carbohydrate conjugates,0.0,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.0,...,441.189819,6.420388,0.390083,174.402759,0.000000,406.022142,7.421322e+06,0.000000e+00,0.000000e+00,2.657055e+06
1,Carbohydrates and carbohydrate conjugates,0.0,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.0,...,150.440454,3.931254,0.165852,8.008466,1.725556,835.646881,2.419239e+06,0.000000e+00,0.000000e+00,0.000000e+00
2,Diterpenoids,0.0,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.0,...,14.008920,0.461140,0.110324,223.861752,0.086405,518.351044,2.988356e+05,0.000000e+00,0.000000e+00,3.519269e+06
3,Diterpenoids,0.0,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.0,...,9.416152,0.000000,0.142362,96.100760,2.059125,5688.164779,0.000000e+00,0.000000e+00,0.000000e+00,1.365321e+06
4,Diterpenoids,0.0,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.0,...,807.499515,27.305535,2.863465,10.306684,0.000000,75.498157,1.725016e+07,0.000000e+00,0.000000e+00,0.000000e+00
5,Terpene lactones,0.0,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.0,...,15.429370,1.707767,0.039210,1.035661,0.676517,712.743784,7.938673e+03,0.000000e+00,0.000000e+00,0.000000e+00
6,Terpene lactones,0.0,3.893345e+04,0.0,0.000000,0.000000,0.000000,0.0,3.413393e+04,0.0,...,2.297116,2.845556,0.018110,1.840802,0.223223,102.085432,3.274544e+04,1.550901e+04,0.000000e+00,1.778840e+04
7,Terpene lactones,0.0,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.0,...,0.000000,0.000000,0.000000,836.916580,0.000000,403.092978,0.000000e+00,0.000000e+00,0.000000e+00,1.339061e+07
8,Terpene glycosides,0.0,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.0,6.562902e+05,0.0,...,720.461091,26.968811,0.788273,8.519395,5.347679,3325.326421,1.164435e+07,2.040203e+05,0.000000e+00,0.000000e+00
9,Terpene lactones,0.0,0.000000e+00,0.0,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.0,...,4.176409,6.776744,0.000000,22.081613,394.637076,13090.178215,0.000000e+00,0.000000e+00,0.000000e+00,1.115718e+05


In [15]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [16]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [17]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [18]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# append accuracy to list
t = []
t.append(clf.score(X_test, y_test))

pickle.dump(clf, open('../model/rf_ng_fs+decomp.sav', 'wb'))

In [19]:
# setting params
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [20]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

pickle.dump(model, open('../model/xgb_ng_fs+decomp.sav', 'wb'))
t.append(model.score(X_test, y_test))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
# define and fit 
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

pickle.dump(gbm, open('../model/lgbm_ng_fs+decomp.sav', 'wb'))
t.append(gbm.score(X_test, y_test))

[1]	valid_0's multi_logloss: 1.28878
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.20847
[3]	valid_0's multi_logloss: 1.13982
[4]	valid_0's multi_logloss: 1.08051
[5]	valid_0's multi_logloss: 1.03088
[6]	valid_0's multi_logloss: 0.98421
[7]	valid_0's multi_logloss: 0.943343
[8]	valid_0's multi_logloss: 0.912084
[9]	valid_0's multi_logloss: 0.886113
[10]	valid_0's multi_logloss: 0.85265
[11]	valid_0's multi_logloss: 0.831805
[12]	valid_0's multi_logloss: 0.804853
[13]	valid_0's multi_logloss: 0.784163
[14]	valid_0's multi_logloss: 0.768032
[15]	valid_0's multi_logloss: 0.752548
[16]	valid_0's multi_logloss: 0.741967
[17]	valid_0's multi_logloss: 0.724925
[18]	valid_0's multi_logloss: 0.718684
[19]	valid_0's multi_logloss: 0.708428
[20]	valid_0's multi_logloss: 0.695186
[21]	valid_0's multi_logloss: 0.689559
[22]	valid_0's multi_logloss: 0.68408
[23]	valid_0's multi_logloss: 0.678473
[24]	valid_0's multi_logloss: 0.67213
[25]	valid_0's multi_

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [22]:
y_pred = gbm.predict(X_test)

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [23]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_ng_fs+decomp.h5')
t.append(model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 52s 140ms/step - loss: 2.1796 - acc: 0.1850 - val_loss: 1.3524 - val_acc: 0.3191
Epoch 2/100
373/373 [==============================] - 0s 780us/step - loss: 1.9082 - acc: 0.2949 - val_loss: 1.3084 - val_acc: 0.4043
Epoch 3/100
373/373 [==============================] - 0s 644us/step - loss: 1.8016 - acc: 0.3029 - val_loss: 1.4017 - val_acc: 0.4255
Epoch 4/100
373/373 [==============================] - 0s 735us/step - loss: 1.7326 - acc: 0.3324 - val_loss: 1.5032 - val_acc: 0.4468
Epoch 5/100
373/373 [==============================] - 0s 568us/step - loss: 1.7848 - acc: 0.3271 - val_loss: 1.5400 - val_acc: 0.4468
Epoch 6/100
373/373 [==============================] - 0s 629us/step - loss: 1.8794 - acc: 0.3592 - val_loss: 1.5212 - val_acc: 0.4468
Epoch 7/100
373/373 [==============================] - 0s 1ms/step - loss: 1.6533 - acc: 0.3378 - val_loss: 1.4773 - val_acc: 0.4681
Epoch 8/100

373/373 [==============================] - 0s 786us/step - loss: 0.6715 - acc: 0.7212 - val_loss: 0.8885 - val_acc: 0.5851
Epoch 62/100
373/373 [==============================] - 0s 603us/step - loss: 0.6727 - acc: 0.7239 - val_loss: 0.8939 - val_acc: 0.5851
Epoch 63/100
373/373 [==============================] - 0s 666us/step - loss: 0.6745 - acc: 0.7051 - val_loss: 0.9000 - val_acc: 0.5957
Epoch 64/100
373/373 [==============================] - 0s 673us/step - loss: 0.6472 - acc: 0.7373 - val_loss: 0.9064 - val_acc: 0.5957
Epoch 65/100
373/373 [==============================] - 0s 476us/step - loss: 0.6652 - acc: 0.7239 - val_loss: 0.9154 - val_acc: 0.5957
Epoch 66/100
373/373 [==============================] - 0s 649us/step - loss: 0.6742 - acc: 0.7158 - val_loss: 0.9279 - val_acc: 0.5957
Epoch 67/100
373/373 [==============================] - 0s 860us/step - loss: 0.6179 - acc: 0.7480 - val_loss: 0.9388 - val_acc: 0.5957
Epoch 68/100
373/373 [==============================] - 0s 56

In [24]:
f

[[0.7797619047619048,
  0.8035714285714286,
  0.8511904761904762,
  0.6309523809523809]]

In [25]:
f.append(t)

In [26]:
pd.DataFrame(
    f, 
    columns=['Random Forest', 'XGBoost', 'LightGBM', 'Keras'], 
    index=['Feature Selection + Decomposition_Positive', 'Feature Selection + Decomposition_Negative']
)

,Random Forest,XGBoost,LightGBM,Keras
Feature Selection + Decomposition_Positive,0.779762,0.803571,0.851190,0.630952
Feature Selection + Decomposition_Negative,0.702128,0.755319,0.765957,0.627660


In [27]:
pd.DataFrame(
    f, 
    columns=['Random Forest', 'XGBoost', 'LightGBM', 'Keras'], 
    index=['Feature Selection + Decomposition_Positive', 'Feature Selection + Decomposition_Negative']
).T.to_csv('../result/Feature Selection + Decomposition.csv')